In [3]:
#!pip install textrazor
#!pip install wordcloud
#!pip install plotly

In [1]:
import textrazor
import pandas as pd

import pyodbc
from sqlalchemy import create_engine, event
import urllib

import numpy as np
import pandas as pd
from os import path
#from PIL import Image
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline

In [8]:
# Local
params = urllib.parse.quote_plus(r'DRIVER={SQL Server};SERVER=JACQUELINEMULLE\LOCAL;DATABASE=ContextExtraction;UID=sa;PWD=Pa$$w0rd123')

# Azure
# params = urllib.parse.quote_plus(r'DRIVER={SQL Server};SERVER=growthpoint.database.windows.net;DATABASE=GrowthPointAzure;UID=growthpoint_sa;PWD=Pa$$w0rd123')

conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)


@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True

In [11]:
API_Key = '05f039ba61f13f6c4356da56229b968d2907407ded0e7d14c6fd8022'
URL = 'https://music.youtube.com/watch?v=Y-Wqna-hC2Y&list=PLtcIogydu_Bocl0B4w5EMFR6oDWsBC5Z0'#'https://azure.microsoft.com/en-us/services/devops/boards/'
textrazor.api_key = API_Key

client = textrazor.TextRazor(extractors=["entities", "topics"])
client.set_cleanup_mode("cleanHTML")
client.set_classifiers(["textrazor_newscodes"])

response = client.analyze_url(URL)

In [12]:
entities = list(response.entities())
entities.sort(key=lambda x: x.relevance_score, reverse=True)
seen = set()

for entity in entities:
     if entity.id not in seen:
        #print(entity.id, entity.relevance_score, entity.confidence_score, entity.freebase_types)
        seen.add(entity.id)

In [13]:
Topic_ID = []
JSON = []
Label = []
Score = []
Wikidata = []
Wikipedia_link = []
URL_Link = []

for topic in response.topics():
    if topic.score > 0.3:
        Topic_ID.append(topic.id)
        JSON.append(topic.json)
        Label.append(topic.label)
        Score.append(topic.score)
        Wikidata.append(topic.wikidata_id)
        Wikipedia_link.append(topic.wikipedia_link)
        URL_Link.append(URL)
        
df_Topic = pd.DataFrame()
df_Topic['Topic_ID'] = Topic_ID
df_Topic['JSON'] = str(JSON)
df_Topic['Label'] = Label
df_Topic['Score'] = Score
df_Topic['Wikidata'] = Wikidata
df_Topic['Wikipedia_Link'] = Wikipedia_link
df_Topic['URL'] = URL_Link

df_Topic.to_csv('Topic.csv')
#df_Topic.to_sql(name='Topic',con=engine, if_exists='append',index=False, chunksize=200)

In [14]:
Category_ID = [] 
Category_Label = [] 
Category_Score = [] 
Category_JSON = []
Classifier_ID = []
URL_Link = []

for category in response.categories():
    Category_ID.append(category.category_id)
    Category_Label.append(category.label)
    Category_Score.append(category.score)
    Category_JSON.append(category.json)
    Classifier_ID.append(category.classifier_id)
    URL_Link.append(URL)

df_Category = pd.DataFrame()
df_Category['ID'] = Category_ID
df_Category['Label'] = Category_Label
df_Category['Score'] = Category_Score
df_Category['JSON'] = Category_JSON
df_Category['Classifier_ID'] = Classifier_ID
df_Category['URL'] = URL_Link

In [15]:
count = 0

TopLevel = []
MiddleLevel = []
BottomLevel = []

for category in df_Category['JSON']:
    category_level = len(df_Category['JSON'][count]['label'].split('>'))
    categories = df_Category['JSON'][count]['label'].split('>')
    
    if category_level == 1:
        TopLevel.append(categories[0])
        MiddleLevel.append(None)
        BottomLevel.append(None)
        
    if category_level == 2:
        TopLevel.append(categories[0])
        MiddleLevel.append(categories[1])
        BottomLevel.append(None)
    
    if category_level == 3:
        TopLevel.append(categories[0])
        MiddleLevel.append(categories[1])
        BottomLevel.append(categories[2])
    
    count+=1

df_Category['TopLevel'] = TopLevel
df_Category['MiddleLevel'] = MiddleLevel
df_Category['BottomLevel'] = BottomLevel

#df_Category['JSON'] = str(df_Category['JSON'])

df_Category.to_csv('Category.csv')
#df_Category.to_sql(name='Category',con=engine, if_exists='replace',index=False)

In [16]:
df_Category

,ID,Label,Score,JSON,Classifier_ID,URL,TopLevel,MiddleLevel,BottomLevel
0,04003000,"economy, business and finance>computing and in...",0.5851,"{'id': 0, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,"economy, business and finance",computing and information technology,None
1,01027000,"arts, culture and entertainment>internet",0.5510,"{'id': 1, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,"arts, culture and entertainment",internet,None
2,04003005,"economy, business and finance>computing and in...",0.5256,"{'id': 2, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,"economy, business and finance",computing and information technology,software
3,13022000,science and technology>it/computer sciences,0.4508,"{'id': 3, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,science and technology,it/computer sciences,None
4,13010000,science and technology>technology (general),0.4508,"{'id': 4, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,science and technology,technology (general),None
5,01002000,"arts, culture and entertainment>architecture",0.4508,"{'id': 5, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,"arts, culture and entertainment",architecture,None
6,01011000,"arts, culture and entertainment>music",0.4411,"{'id': 6, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,"arts, culture and entertainment",music,None
7,01021000,"arts, culture and entertainment>entertainment ...",0.4411,"{'id': 7, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,"arts, culture and entertainment",entertainment (general),None
8,13000000,science and technology,0.4056,"{'id': 8, 'classifierId': 'textrazor_newscodes...",textrazor_newscodes,https://music.youtube.com/watch?v=Y-Wqna-hC2Y&...,science and technology,None,None


In [19]:
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='JacquiM', api_key='IZDhiKu8RSqJrCQSLNyd')

trace1 = go.Bar(
    x = df_Category.TopLevel,
    y = df_Category.Score,
    name='Top Level Category'
)
trace2 = go.Bar(
    x = df_Category.MiddleLevel,
    y = df_Category.Score,
    name='Middle Level Category'
)
trace3 = go.Bar(
    x = df_Category.BottomLevel,
    y = df_Category.Score,
    name='Bottom Level Category'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Category')

In [18]:
# Start with one review:
text = str(df_Category.Label)

# Create and generate a word cloud image:
wordcloud = WordCloud(background_color='white').generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

NameError: name 'wordcloud' is not defined